### ETL, Postgres DB
This is a test notebook for ETL on postgres SQL, a dumped SQL from after ELK flow. 
IP: 192.168.7.114, hua's test VM, CentOS. Please make sure the docker_db_1 container is running and some pcap files has been processed manually or automatically.

In [1]:
from sshtunnel import SSHTunnelForwarder
import psycopg2
import sys

/Users/ustc-weihua/anaconda/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
# https://sshtunnel.readthedocs.io/en/latest/
def test_conn(hostip = "192.168.7.114"):
    PORT=5432
    with SSHTunnelForwarder(
        hostip,
        ssh_username="root",
        ssh_password="Helios12$",
        remote_bind_address=('localhost', PORT),
        local_bind_address=('localhost', PORT)
    ):
        #Define our connection string
        conn_string = "host='localhost' dbname='helios' user='helios' password='helios'"
 
        # print the connection string we will use to connect
        print("Connecting to database\n ->%s" % (conn_string))
 
        # get a connection, if a connect cannot be made an exception will be raised here
        conn = psycopg2.connect(conn_string)
 
        # conn.cursor will return a cursor object, you can use this cursor to perform queries
        cursor = conn.cursor()
        print("Connected!\n")

        cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
        print(cursor.fetchall())

test_conn()

Connecting to database
 ->host='localhost' dbname='helios' user='helios' password='helios'
Connected!

[('http',), ('conn',), ('files',), ('dns',)]


In [3]:
import numpy as np
import pandas as pd

In [4]:
# function to print the column name for the table
def db_print_colnames(dbhostip, tablename):
    PORT=5432
    with SSHTunnelForwarder(
        dbhostip,
        ssh_username="root",
        ssh_password="Helios12$",
        remote_bind_address=('localhost', PORT),
        local_bind_address=('localhost', PORT)
    ):
        #Define our connection string
        conn_string = "host='localhost' dbname='helios' user='helios' password='helios'"
 
        # print the connection string we will use to connect
        #print("Connecting to database\n ->%s" % (conn_string))
 
        # get a connection, if a connect cannot be made an exception will be raised here
        conn = psycopg2.connect(conn_string)
 
        # conn.cursor will return a cursor object, you can use this cursor to perform queries
        cursor = conn.cursor()
        
        # execute our Query
        sqlquery = "select column_name from information_schema.columns where table_name='" + tablename + "'"
        cursor.execute(sqlquery)
        # retrieve the records from the database
        colnames = cursor.fetchall()
        return colnames

for tablename in ['http', 'conn', 'files', 'dns']:
    print(tablename)
    print( [x[0] for x in db_print_colnames("192.168.7.114", tablename)] )

http
['id.orig_p', 'method', 'response_body_len', 'uid', 'orig_mime_types', 'info_code', 'referrer', 'filename', 'user_agent', 'info_msg', 'ts', 'resp_mime_types', 'id.resp_h', 'host', 'id.resp_p', 'username', 'status_msg', 'request_body_len', 'id.orig_h', 'uri', 'status_code', 'kafka_offset']
conn
['service', 'duration', 'resp_bytes', 'orig_bytes', 'history', 'ts', 'resp_cc', 'uid', 'orig_cc', 'conn_state', 'kafka_offset']
files
['fuid', 'filename', 'tx_hosts', 'conn_uids', 'ts', 'source', 'rx_hosts', 'mime_type', 'extracted', 'kafka_offset']
dns
['qclass', 'qtype', 'AA', 'rcode', 'RA', 'query', 'TTL', 'ts', 'TC', 'RD', 'answers', 'qtype_name', 'uid', 'rcode_name', 'rejected', 'Z', 'qclass_name', 'kafka_offset']


In [5]:
# function to convert the table in the postgres database to pandas dataframe
def db_to_df(dbhostip, tablename):
    PORT=5432
    with SSHTunnelForwarder(
        dbhostip,
        ssh_username="root",
        ssh_password="Helios12$",
        remote_bind_address=('localhost', PORT),
        local_bind_address=('localhost', PORT)
    ):
        #Define our connection string
        conn_string = "host='localhost' dbname='helios' user='helios' password='helios'"
 
        # print the connection string we will use to connect
        print("Connecting to database\n ->%s" % (conn_string))
 
        # get a connection, if a connect cannot be made an exception will be raised here
        conn = psycopg2.connect(conn_string)
 
        # conn.cursor will return a cursor object, you can use this cursor to perform queries
        cursor = conn.cursor()
        
        # get some records
        # execute our Query
        sqlquery = "select * from " + tablename + " limit 10"
        cursor.execute(sqlquery)
        # retrieve the records from the database
        records = cursor.fetchall()
        cursor.execute("select column_name from information_schema.columns where table_name='dns'")
        #print (cursor.fetchall())
        
        # get the column names
        # execute our Query
        sqlquery = "select column_name from information_schema.columns where table_name='" + tablename + "'"
        cursor.execute(sqlquery)
        # retrieve the records from the database
        colnames = [x[0] for x in cursor.fetchall()]
        
        df = pd.DataFrame(records, columns=colnames)
        return df
        
#mytest_df = db_to_df("192.168.7.114", "files")
#mytest_df.head()

### table investigation, http

column name | description
----------- | -----------
id.orig_p   | 
method      | GET, POST
response_body_len | the length of the packet
uid | hashed user id, to pretect the privacy
orig_mime_types | ?
info_code  | ?
referrer   | ?
filename   | the name of downloaded file?
user_agent | value of user agent header
info_msg   | ?
ts         | ?
resp_mime_types | ?
id.resp_h | ?
host | ?
id.resp_p | ?
username  | ?
status_msg |
request_body_len | ?
id.orig_h | ?
uri | ?
status_code | ?
kafka_offset | ?

In [6]:
http_df = db_to_df("192.168.7.114", "http")
http_df.head()

Connecting to database
 ->host='localhost' dbname='helios' user='helios' password='helios'


,id.orig_p,method,response_body_len,uid,orig_mime_types,info_code,referrer,filename,user_agent,info_msg,...,id.resp_h,host,id.resp_p,username,status_msg,request_body_len,id.orig_h,uri,status_code,kafka_offset
0,49649,GET,95148,CJUSMA13Ss6nNrRlS8,,None,None,,Wget/1.19.4 (darwin17.3.0),,...,35.190.55.229,www.wenxuecity.com,80,,OK,0,10.0.1.73,/,200,0
1,62321,GET,4,CEkr1d1VA6KreLgy4,,None,None,,OSCE;12.0;1775;dd9c3434-f845-4bd8-86f7-cde1f48...,,...,192.168.3.111,Officescan,8080,,OK,0,192.168.3.61,/ss/q/?BF=1748000400&LCRC=AB795571F586FB4A06AF...,200,1
2,53051,GET,4,Cdaic71JEnlnfmg3Wl,,None,None,,OSCE;12.0;1775;dd9c3434-f845-4bd8-86f7-cde1f48...,,...,192.168.3.111,Officescan,8080,,OK,0,192.168.3.121,/ss/q/?BF=1748000400&LCRC=AB795571F586FB4A07AF...,200,2
3,38143,GET,2147,CK5k1hNiD3nUR7WE2,,None,None,,Mozilla/5.0 (X11; Linux x86_64; rv:38.0) Gecko...,,...,172.16.1.29,detik.com,3128,,Proxy Authentication Required,0,192.168.3.183,http://detik.com/,407,3
4,38144,GET,183,ChGsLH1xHozx8NddRa,,None,None,,Mozilla/5.0 (X11; Linux x86_64; rv:38.0) Gecko...,,...,172.16.1.29,detik.com,3128,,Moved Permanently,0,192.168.3.183,http://detik.com/,301,4


### table investigation, conn

column name | description
----------- | -----------
service | 
duration | 
resp_bytes |
orig_bytes | 
history | 
ts |
resp_cc | 
uid | 
orig_cc | 
conn_state | 
kafka_offset |

In [7]:
conn_df = db_to_df("192.168.7.114", "conn")
conn_df.head()

Connecting to database
 ->host='localhost' dbname='helios' user='helios' password='helios'


,service,duration,resp_bytes,orig_bytes,history,ts,resp_cc,uid,orig_cc,conn_state,kafka_offset
0,http,0.424982,96029.0,148.0,ShADadFf,1.522387e+09,,CJUSMA13Ss6nNrRlS8,,SF,0
1,None,0.699986,279.0,910.0,ShADd,1.522387e+09,,CRpUhA3MhlkJys7pzb,,S1,1
2,dns,NaN,NaN,NaN,D,1.521699e+09,,CuqCui33BcocOvGBej,,S0,2
3,None,NaN,NaN,NaN,S,1.521699e+09,,Cg4lLc2cgSm1rBILn2,,S0,3
4,dns,0.000448,103.0,39.0,Dd,1.521699e+09,,CbAlgMdwf8DrlbHQ5,,SF,4


### table investigation, files

column name | description
----------- | -----------
fuid      | 
filename  | 
tx_hosts  | 
conn_uids | 
ts        | 
source    | 
rx_hosts  | 
mime_type | 
extracted | 
kafka_offset |

In [8]:
files_df = db_to_df("192.168.7.114", "files")
files_df.head()

Connecting to database
 ->host='localhost' dbname='helios' user='helios' password='helios'


,fuid,filename,tx_hosts,conn_uids,ts,source,rx_hosts,mime_type,extracted,kafka_offset
0,FL0Tap4fGIaZwniIVc,,{35.190.55.229},{CJUSMA13Ss6nNrRlS8},1.522387e+09,HTTP,{10.0.1.73},text/html,HTTP-FL0Tap4fGIaZwniIVc.html,0
1,FB52dEaGdh7RRUbu7,,{192.168.3.111},{CEkr1d1VA6KreLgy4},1.521700e+09,HTTP,{192.168.3.61},None,,1
2,FOS6qT17ymqq7JNUI6,,{192.168.3.111},{Cdaic71JEnlnfmg3Wl},1.521701e+09,HTTP,{192.168.3.121},None,,2
3,FGIt8E4nXBFXCrB16b,,{172.16.1.29},{CK5k1hNiD3nUR7WE2},1.521701e+09,HTTP,{192.168.3.183},text/html,HTTP-FGIt8E4nXBFXCrB16b.html,3
4,FWer7XRB7gRdOHRO5,,{172.16.1.29},{ChGsLH1xHozx8NddRa},1.521701e+09,HTTP,{192.168.3.183},text/html,HTTP-FWer7XRB7gRdOHRO5.html,4


### table investigation, dns

column name | description
----------- | -----------
qclass | 
qtype | 
AA | 
rcode | 
RA | 
query | 
TTL | 
ts | 
TC | 
RD | 
answers | 
qtype_name | 
uid | 
rcode_name | 
rejected | 
Z | 
qclass_name | 
kafka_offset |

In [9]:
dns_df = db_to_df("192.168.7.114", "dns")
dns_df.head()

Connecting to database
 ->host='localhost' dbname='helios' user='helios' password='helios'


,qclass,qtype,AA,rcode,RA,query,TTL,ts,TC,RD,answers,qtype_name,uid,rcode_name,rejected,Z,qclass_name,kafka_offset
0,1,1,False,0.0,True,mirrorlist.centos.org,None,1.521699e+09,False,True,"{212.69.166.138,216.176.179.218,67.219.148.138...",A,CbAlgMdwf8DrlbHQ5,NOERROR,False,0,C_INTERNET,0
1,1,1,False,NaN,False,mirrorlist.centos.org,None,1.521699e+09,False,True,None,A,CuqCui33BcocOvGBej,None,False,0,C_INTERNET,1
2,1,1,False,NaN,False,mirror.centos.org,None,1.521699e+09,False,True,None,A,CzuAHw3yNAyrqAQhfg,None,False,0,C_INTERNET,2
3,1,1,False,2.0,False,corefouat1,None,1.521699e+09,False,True,None,A,CTO8ti3KtEGmFrHkM7,SERVFAIL,True,0,C_INTERNET,3
4,1,28,False,2.0,False,corefouat1,None,1.521699e+09,False,True,None,AAAA,CTO8ti3KtEGmFrHkM7,SERVFAIL,True,0,C_INTERNET,4


### The end of this notebook